In [1]:
import pandas as pd
import numpy as np
import scipy as sp

%matplotlib inline

In [2]:
import sklearn
from sklearn import preprocessing

In [93]:
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn.neighbors import DistanceMetric 

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Cargo todos los CSV

In [28]:
postulantes_educacion = pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv'))

In [29]:
postulantes_genero_y_edad = pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))

In [3]:
vistas = pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv',nrows=10000)
vistas = vistas.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_3_vistas.csv',nrows=10000))
vistas = vistas.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_3_vistas.csv',nrows=10000))

In [1]:
postulaciones = pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv',nrows=10000)
postulaciones = postulaciones.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv',nrows=10000))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')

NameError: name 'pd' is not defined

In [32]:
avisos_online = pd.read_csv('data/datos_navent_fiuba/fiuba_5_avisos_online.csv',nrows=10000)
avisos_online = avisos_online.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv',nrows=10000))
# no tiene avisos online pd.read_csv('data/fiuba_desde_15_abril/')

In [30]:
avisos_detalle = pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv',nrows=10000)
#avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv',nrows=10000))
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv',nrows=10000))
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv',nrows=10000))

In [31]:
test_final = pd.read_csv('data/test_final_100k.csv')

- Limpio el DF *postulantes_educacion*

In [97]:
postulantes_educacion = postulantes_educacion.reset_index().drop(columns='index')


In [98]:
postulantes_educacion.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [99]:
postulantes_educacion.isnull().sum()

idpostulante    0
nombre          0
estado          0
dtype: int64

In [100]:
postulantes_educacion.isna().sum()

idpostulante    0
nombre          0
estado          0
dtype: int64

- Limpio el DF *postulantes_genero_y_edad*

In [101]:
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop(columns='index')

In [102]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [103]:
postulantes_genero_y_edad.shape

(780020, 3)

In [104]:
postulantes_genero_y_edad.isna().sum()

idpostulante           0
fechanacimiento    34273
sexo                   0
dtype: int64

In [105]:
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    34273
sexo                   0
dtype: int64

In [33]:
postulantes_genero_y_edad = postulantes_genero_y_edad[-postulantes_genero_y_edad['fechanacimiento'].isna()]

In [73]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()

In [35]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']


,idpostulante,fechanacimiento,sexo
168364,0zP6MJa,1985-08-16,FEM
26817,0zP6MJa,1989-08-16,FEM


In [36]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

,idpostulante,fechanacimiento,sexo
284748,0zPvY49,1991-03-26,MASC
180194,0zPvY49,1991-03-26,FEM


In [74]:
#Dejo la ultima aparicion del duplicado, supongo que es el update más reciente
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')

In [75]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

,idpostulante,fechanacimiento,sexo
26817,0zP6MJa,1989-08-16,FEM


In [76]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

,idpostulante,fechanacimiento,sexo
180194,0zPvY49,1991-03-26,FEM


In [113]:
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop(columns='index')

- Limpio el DF *vistas*

In [5]:
vistas.head()
#vistas = vistas.reset_index().drop(columns='index')

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [6]:
vistas.isna().sum()

idAviso         0
timestamp       0
idpostulante    0
dtype: int64

In [118]:
vistas.isnull().sum()

idAviso         0
timestamp       0
idpostulante    0
dtype: int64

In [119]:
vistas.duplicated().sum()

0

- Limpio el DF *postulaciones*

In [123]:
#postulaciones = postulaciones.reset_index().drop(columns='index')

In [7]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [126]:
postulaciones.isna().sum()

idaviso             0
idpostulante        0
fechapostulacion    0
dtype: int64

In [127]:
postulaciones.isnull().sum()

idaviso             0
idpostulante        0
fechapostulacion    0
dtype: int64

In [128]:
postulaciones.duplicated().sum()

3402

In [129]:
postulaciones = postulaciones.drop_duplicates(keep='last')

In [130]:
postulaciones = postulaciones.reset_index().drop(columns='index')

In [131]:
postulaciones.shape

(16598, 3)

- Limpio el DF *avisos_online*

In [132]:
avisos_online = avisos_online.reset_index().drop(columns='index')

In [133]:
avisos_online.head()

,idaviso
0,1112355872
1,1112335374
2,1112374842
3,1111984070
4,1111822480


In [134]:
avisos_online.isna().sum()

idaviso    0
dtype: int64

In [135]:
avisos_online.isnull().sum()

idaviso    0
dtype: int64

In [136]:
avisos_online = avisos_online.drop_duplicates(keep='last')

In [137]:
avisos_online = avisos_online.reset_index().drop(columns='index')

- Limpio el DF *avisos_detalle*

In [138]:
avisos_detalle = avisos_detalle.reset_index().drop(columns='index')

In [40]:
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [140]:
avisos_detalle.shape

(20338, 11)

In [41]:
avisos_detalle['idaviso'].duplicated().sum()

1744

In [42]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')

In [43]:
avisos_detalle = avisos_detalle.reset_index().drop(columns='index')

In [8]:
vistas = vistas.rename(columns={'idAviso': 'idaviso'})

In [19]:
df_se_postulo = pd.merge(vistas,postulaciones,how="left",left_on=["idaviso","idpostulante"],right_on=["idaviso","idpostulante"])
df_se_postulo

,idaviso,timestamp,idpostulante,fechapostulacion
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z,NaN
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR,NaN
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd,NaN
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv,NaN
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP,NaN
5,1112355613,2018-02-23T13:38:23.467-0500,rmdWApz,NaN
6,1112346704,2018-02-23T13:38:27.130-0500,Dr6e6Xl,NaN
7,1112282610,2018-02-23T13:38:27.670-0500,A3R9GWl,NaN
8,1112342544,2018-02-23T13:38:31.601-0500,8M24rbM,NaN
9,1112358408,2018-02-23T13:38:32.977-0500,YjrD1ZZ,NaN


In [20]:
df_se_postulo = df_se_postulo.rename(columns={'fechapostulacion':'se_postulo'})

In [22]:
df_se_postulo.drop(columns="timestamp",inplace=True)

In [23]:
df_se_postulo['se_postulo']=df_se_postulo['se_postulo'].fillna(0)

In [25]:
df_se_postulo.loc[df_se_postulo['se_postulo'] != 0, 'se_postulo'] = 1

In [60]:
df_se_postulo['se_postulo'].value_counts()

0    29948
1       58
Name: se_postulo, dtype: int64

In [27]:
df_se_postulo.head()

,idaviso,idpostulante,se_postulo
0,1111780242,YjVJQ6Z,0
1,1112263876,BmVpYoR,0
2,1112327963,wVkBzZd,0
3,1112318643,OqmP9pv,0
4,1111903673,DrpbXDP,0


In [78]:
postulantes_educacion.loc[postulantes_educacion['idpostulante'] == 'NdJl']

,idpostulante,nombre,estado
1237,NdJl,Posgrado,En Curso
1743,NdJl,Universitario,Graduado


In [77]:
postulantes_educacion.drop_duplicates(inplace=True,keep="last")

In [79]:
postulantes = pd.merge(postulantes_educacion,postulantes_genero_y_edad, on ="idpostulante",how="inner")
postulantes.shape

(682315, 5)

In [80]:
le = preprocessing.LabelEncoder()
le.fit(postulantes['sexo'])
postulantes['sexo'] = le.transform(postulantes['sexo'])
le.fit(postulantes['estado'])
postulantes['estado'] = le.transform(postulantes['estado'])
le.fit(postulantes['nombre'])
postulantes['nombre'] = le.transform(postulantes['nombre'])
postulantes.head()

,idpostulante,nombre,estado,fechanacimiento,sexo
0,8BkL,6,1,1976-07-21,0
1,8BkL,6,2,1976-07-21,0
2,8BkL,3,2,1976-07-21,0
3,8BkL,1,2,1976-07-21,0
4,1d2B,6,1,1976-02-28,1


In [81]:
postulantes = postulantes[(postulantes['fechanacimiento'].str.startswith('19'))| (postulantes['fechanacimiento'].str.startswith('20'))]
postulantes['fechanacimiento'] = pd.to_datetime('today').year-pd.to_datetime(postulantes['fechanacimiento']).dt.year


In [182]:
test_final.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [82]:
train_test_merge = pd.merge(df_se_postulo, postulantes, on="idpostulante", how="inner")
train_test_merge

,idaviso,idpostulante,se_postulo,nombre,estado,fechanacimiento,sexo
0,1111780242,YjVJQ6Z,0,4,2,27,1
1,1111780242,YjVJQ6Z,0,6,0,27,1
2,1112263876,BmVpYoR,0,6,1,29,1
3,1112356639,BmVpYoR,0,6,1,29,1
4,1112355153,BmVpYoR,0,6,1,29,1
5,1112327963,wVkBzZd,0,6,1,26,1
6,1112327963,wVkBzZd,0,5,0,26,1
7,1112327963,wVkBzZd,0,5,1,26,1
8,1112327963,wVkBzZd,0,4,2,26,1
9,1112327963,wVkBzZd,0,6,0,26,1


In [84]:
train_test = pd.merge(train_test_merge,avisos_detalle,on="idaviso",how="inner")

In [85]:
train_test.head()

,idaviso,idpostulante,se_postulo,nombre,estado,fechanacimiento,sexo,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111780242,YjVJQ6Z,0,4,2,27,1,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
1,1111780242,YjVJQ6Z,0,6,0,27,1,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
2,1111780242,dYjRpMD,0,4,2,24,0,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
3,1111780242,dYjRpMD,0,4,2,24,0,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm
4,1111780242,wVopPXl,0,4,2,49,1,1,"OPERARIO PARA DEPOSITO, CONFECCION DE PEDIDOS,...",<p>SE NECESITA OPERIARIO PARA DEPOSITO CONFECC...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación de producción,dm


In [86]:
train_test_clean = train_test

In [87]:
train_test_clean.drop(columns=["titulo","descripcion","ciudad","mapacalle","idpais"],inplace=True)

In [88]:
#esto es una negrada
train_test_clean = train_test_clean.fillna(0)

In [89]:
le.fit(train_test_clean['nivel_laboral'])
train_test_clean['nivel_laboral'] = le.transform(train_test_clean['nivel_laboral'])
le.fit(train_test_clean['tipo_de_trabajo'])
train_test_clean['tipo_de_trabajo'] = le.transform(train_test_clean['tipo_de_trabajo'])
le.fit(train_test_clean['nombre_area'])
train_test_clean['nombre_area'] = le.transform(train_test_clean['nombre_area'])
le.fit(train_test_clean['denominacion_empresa'])
train_test_clean['denominacion_empresa'] = le.transform(train_test_clean['denominacion_empresa'])
le.fit(train_test_clean['nombre_zona'])
train_test_clean['nombre_zona'] = le.transform(train_test_clean['nombre_zona'])
train_test_clean

,idaviso,idpostulante,se_postulo,nombre,estado,fechanacimiento,sexo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111780242,YjVJQ6Z,0,4,2,27,1,3,1,5,118,1188
1,1111780242,YjVJQ6Z,0,6,0,27,1,3,1,5,118,1188
2,1111780242,dYjRpMD,0,4,2,24,0,3,1,5,118,1188
3,1111780242,dYjRpMD,0,4,2,24,0,3,1,5,118,1188
4,1111780242,wVopPXl,0,4,2,49,1,3,1,5,118,1188
5,1111780242,wVopPXl,0,4,2,49,1,3,1,5,118,1188
6,1111780242,wVoOPoL,0,4,2,34,1,3,1,5,118,1188
7,1111780242,wVoOPoL,0,6,2,34,1,3,1,5,118,1188
8,1111780242,wVoOPoL,0,4,2,34,1,3,1,5,118,1188
9,1111780242,wVoOPoL,0,6,2,34,1,3,1,5,118,1188


In [97]:
#knn?
# X will have the numeric data, Y will have the target class
X = np.array(train_test_clean.ix[:, 3:12]) 
y = np.array(train_test_clean['se_postulo']) 
# Split into train and test, use only 10% as test (15 flowers)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=7506)
# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=4)
# FIT (this does nothing in normal KNN)
knn.fit(X_train, y_train)
# predict the response 
pred = knn.predict(X_test)
# evaluate accuracy

print accuracy_score(y_test, pred)
print pred
print X_test
print y_test
print pred == y_test


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


0.9984349258649093
[0 0 0 ... 0 0 0]
[[   6    2   33 ...    3   39  861]
 [   5    2   57 ...    5  124  206]
 [   6    2   43 ...    4  140  704]
 ...
 [   6    1   22 ...    3   13  105]
 [   5    0   29 ...    3   96 1172]
 [   4    2   26 ...    3   22 1004]]
[0 0 0 ... 0 0 0]
[ True  True  True ...  True  True  True]
